In [13]:
import csv
import ast
import os
import requests
from dotenv import load_dotenv
import mysql.connector
import ssl
import pymysql

# Initialize an empty array to store the data
# data = []

# Open the CSV file

load_dotenv()

DELIMITER = "<BRK>"
                                
counter = -1

In [8]:
# Adding this flag so we don't update every time 
update_songs_flag = False

conn = pymysql.connect(
    user=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=int(os.getenv('DB_PORT')),
    database=os.getenv('DB_NAME'),
    ssl={'ca': './ca-certificate.crt'}
)

cursor = conn.cursor()

if update_songs_flag == True: 
    with open('final_tracks.csv', mode='r') as file:
        csv_reader = csv.reader(file)

        for row in csv_reader:
            if counter >= 0 and counter > last_committed:
                index = counter
                name = row[1]
                artists = ast.literal_eval(row[2])
                artists_str = DELIMITER.join(artists)
                song_id = row[3]
                popularity = row[4] 
                artist_ids = ast.literal_eval(row[8])
                artist_ids_str = DELIMITER.join(artist_ids)
                playlist_ids = ast.literal_eval(row[9])
                num_playlists = len(playlist_ids)
                playlist_ids_str = DELIMITER.join(playlist_ids)
                
                print(str(index) + ":", name, "-", artists_str)

                query = """
                    INSERT INTO CS_229_SONGS_ALL (SONG_NUM, NAME, ARTISTS, SONG_ID, POPULARITY, ARTIST_ID, PLAYLIST_IDS, NUM_PLAYLISTS)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """

                cursor.execute(query, (index, name, artists_str, song_id, popularity, artist_ids_str, playlist_ids_str, num_playlists))
                conn.commit()

            last_committed = counter
            counter += 1

conn.close()

In [15]:
import os
import pymysql
from dotenv import load_dotenv

playlist_set = set()
songs_set = set()
num_songs = 1000
# Load environment variables from .env file
load_dotenv()

# Establish a connection to the database
conn = pymysql.connect(
    user=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=int(os.getenv('DB_PORT')),
    database=os.getenv('DB_NAME')
)

try:
    with conn.cursor() as cursor:
        # SQL query to find the top 10,000 songs based on NUM_PLAYLISTS
        query = f"""
            SELECT SONG_NUM, NAME, NUM_PLAYLISTS, PLAYLIST_IDS
            FROM CS_229_SONGS_ALL
            ORDER BY NUM_PLAYLISTS DESC
            LIMIT {num_songs}
        """
        cursor.execute(query)
        
        # Fetch all results
        top_songs = cursor.fetchall()

        # Print results
        for song in top_songs:
            playlist_ids_list = song[3].split(DELIMITER)
            playlist_set.update(set(playlist_ids_list))
            songs_set.add(song[0])
            print(f"Song Number: {song[0]}, Name: {song[1]}, Number of Playlists: {song[2]}")

finally:
    # Close the connection
    conn.close()

Song Number: 4970, Name: Sweater Weather, Number of Playlists: 97
Song Number: 4670, Name: Smells Like Teen Spirit, Number of Playlists: 88
Song Number: 7109, Name: Mr. Brightside, Number of Playlists: 88
Song Number: 4113, Name: Riptide, Number of Playlists: 83
Song Number: 5079, Name: The Less I Know The Better, Number of Playlists: 75
Song Number: 7119, Name: Pumped Up Kicks, Number of Playlists: 73
Song Number: 5505, Name: Sweet Child O' Mine, Number of Playlists: 71
Song Number: 4406, Name: Ophelia, Number of Playlists: 70
Song Number: 24224, Name: Sweet Home Alabama, Number of Playlists: 69
Song Number: 13239, Name: Heat Waves, Number of Playlists: 67
Song Number: 1162, Name: Everybody Wants To Rule The World, Number of Playlists: 67
Song Number: 1848, Name: Space Song, Number of Playlists: 66
Song Number: 46946, Name: Welcome To The Jungle, Number of Playlists: 66
Song Number: 11513, Name: Livin' On A Prayer, Number of Playlists: 66
Song Number: 11455, Name: Tongue Tied, Number 

In [10]:
print(str(len(playlist_set)))

2891


In [11]:
load_dotenv()

# This dict maps playlist to the songs in those playlists 
playlist_songs_dict = {}

for playlist in playlist_set: 
    playlist_songs_dict[playlist] = []

conn = pymysql.connect(
    user=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=int(os.getenv('DB_PORT')),
    database=os.getenv('DB_NAME')
)

try:
    with conn.cursor() as cursor:
        query = """
            SELECT SONG_NUM, PLAYLIST_IDS
            FROM CS_229_SONGS_ALL
            ORDER BY NUM_PLAYLISTS DESC
        """
        cursor.execute(query)

        all_songs = cursor.fetchall()

        for song in all_songs:
            add_song_flag = False
            for playlist in song[1].split(DELIMITER): 
                if playlist in playlist_set: 
                    playlist_songs_dict[playlist].append(song[0])
                    add_song_flag = True

            if add_song_flag: 
                songs_set.add(song[0])

finally:
    # Close the connection
    conn.close()

In [12]:
print("# of songs in the database is:", str(len(songs_set)))

# of songs in the database is: 108338


In [13]:
# Making co-occurrence matrix 

from scipy.sparse import lil_matrix, csr_matrix
import numpy as np

# Our vocab is the songs_set 
# Map song names to indexes 
song_to_index = {song: index for index, song in enumerate(list(songs_set))}

cooccurrence_matrix = lil_matrix((len(songs_set), len(songs_set)), dtype=int)

playlist_counter = 0
for playlist in playlist_set: 
    # Indexes as row, col in matrix to update, mapped from the songs associated with the playlist
    indexes = [song_to_index[song] for song in playlist_songs_dict[playlist]]
    for i in range(len(indexes)):
        for j in range(i + 1, len(indexes)): 
            cooccurrence_matrix[indexes[i], indexes[j]] += 1
            cooccurrence_matrix[indexes[j], indexes[i]] += 1
    playlist_counter += 1
    if playlist_counter % 100 == 0: 
        print("Finished processing playlist #" + str(playlist_counter))

Finished processing playlist #100
Finished processing playlist #200
Finished processing playlist #300
Finished processing playlist #400
Finished processing playlist #500
Finished processing playlist #600
Finished processing playlist #700
Finished processing playlist #800
Finished processing playlist #900
Finished processing playlist #1000
Finished processing playlist #1100
Finished processing playlist #1200
Finished processing playlist #1300
Finished processing playlist #1400
Finished processing playlist #1500
Finished processing playlist #1600
Finished processing playlist #1700
Finished processing playlist #1800
Finished processing playlist #1900
Finished processing playlist #2000
Finished processing playlist #2100
Finished processing playlist #2200
Finished processing playlist #2300
Finished processing playlist #2400
Finished processing playlist #2500
Finished processing playlist #2600
Finished processing playlist #2700
Finished processing playlist #2800


In [26]:
# Sanity checks on the co-occurrence matrix 
cooccurrence_matrix = cooccurrence_matrix.tocsr()
print("Co-occurrence matrix shape:", cooccurrence_matrix.shape)
print("Number of non-zero entries:", cooccurrence_matrix.nnz)
print("Number of zero entries:", str(108338 ** 2 - cooccurrence_matrix.nnz))

row_sums = cooccurrence_matrix.sum(axis=1)
column_sums = cooccurrence_matrix.sum(axis=0)

row_sums = np.array(row_sums).flatten()
column_sums = np.array(column_sums).flatten()

print("Row sums (first 10):", row_sums[:10])
print("Column sums (first 10):", column_sums[:10])


Co-occurrence matrix shape: (108338, 108338)
Number of non-zero entries: 18575560
Number of zero entries: 11718546684
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
# TODO: Initialize num_songs x embedding_dim 2D matrix of embeddings
# TODO: Start learning. Our loss function is the sum for each song pair dot product of two song embeddings - their co-occurrences (maybe log'd) squared